In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [138]:
player_data = pd.read_csv('..\\Consolidated Data\\Player_Attribute_Data.csv')
team_rankings = pd.read_csv('..\\Consolidated Data\\Team Rankings_corrected.csv')
spend_df = pd.read_csv('..\\Consolidated Data\\Average_Transfer_Budget.csv')

In [139]:
team_rankings=team_rankings.drop(['Unnamed: 0'],axis=1)
player_df = player_data[['sofifa_id','club', 'overall','position', 'section', 'role']]
combo_df = pd.merge(player_df,team_rankings,how='inner',left_on='club',right_on='Team')
combo_df=combo_df.drop(['sofifa_id','club','Team'],axis=1)
role_df = combo_df.groupby(['role','Tier']).mean()
pos_df = combo_df.groupby(['position','Tier']).mean()
role_df.reset_index(inplace = True)
pos_df.reset_index(inplace = True)

In [140]:
def club_level(club):
    club_tier = (team_rankings[team_rankings['Team'] == club]['Tier']).values[0]
    club_df = player_data[player_data['club']==club][['overall','section','role']]
    club_df = club_df.groupby(['role']).mean()
    club_df = club_df.reset_index()
    upper_tier_df = role_df[role_df['Tier']==(club_tier-1)]
    upper_tier_df = upper_tier_df.reset_index()
    upper_tier_df = upper_tier_df.drop(['index','Tier'],axis=1)
    combo_df = pd.merge(upper_tier_df,club_df,how='inner',on='role')
    combo_df.columns = ['Role','Upper_Tier_Rating','Club_Rating']
    combo_df['Rating_Gap'] = combo_df['Upper_Tier_Rating'] - combo_df['Club_Rating']
#     return combo_df
    role_focus = combo_df[combo_df['Rating_Gap']==combo_df['Rating_Gap'].values.max()]['Role'].values[0]
    return role_focus

In [141]:
def club_gap(club):
    club_tier = (team_rankings[team_rankings['Team'] == club]['Tier']).values[0]
    club_df = player_data[player_data['club']==club][['overall','section','position']]
    club_df = club_df.groupby(['position']).mean()
    club_df = club_df.reset_index()
    upper_tier_df = pos_df[pos_df['Tier']==(club_tier-1)]
    upper_tier_df = upper_tier_df.reset_index()
    upper_tier_df = upper_tier_df.drop(['index','Tier'],axis=1)
    combo_df = pd.merge(upper_tier_df,club_df,how='inner',on='position')
    combo_df.columns = ['Position','Upper_Tier_Rating','Club_Rating']
    combo_df['Rating_Gap'] = combo_df['Upper_Tier_Rating'] - combo_df['Club_Rating']
#     return combo_df
    pos_focus = combo_df[combo_df['Rating_Gap']==combo_df['Rating_Gap'].values.max()]['Position'].values[0]
    return pos_focus

In [142]:
role_to_fill=club_level('Liverpool')

In [143]:
pos_to_fill = club_gap('Liverpool')

In [144]:
player_df = player_data[['sofifa_id','short_name','long_name','club', 'overall','position', 'section', 'role','age','value_eur']]
player_df['value_in_mil'] = player_df['value_eur']/1000000

c:\users\akhil_s_bhat\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [157]:
def rec_player_by_role(club,pct):
    spend_limit = (spend_df[spend_df['Team']==club]['Total_Arrivals'].values[0]) * pct
    clean_df = player_df[player_df['club'] != club]
    limit_df=clean_df[clean_df['value_in_mil']<spend_limit]
    age_df = limit_df[['club','age']].groupby('club').mean()
    age_df['age'] = round(age_df['age'],2)
    age_df = age_df.reset_index()
    target_age = player_df[player_df['club'] == club]
    target_age=target_age['age'].values[0]
    role_df = limit_df[clean_df['role']==role_to_fill]
    age_range_df = role_df[(role_df['age']>=(target_age-1.5)) & (role_df['age']<=(target_age+1.5))]
    return(age_range_df.head(1)['short_name'].values[0])

In [158]:
rec_player_by_role('Liverpool',.6)

c:\users\akhil_s_bhat\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


'Thiago'

In [164]:
def rec_player_by_pos(club,pct):
    spend_limit = (spend_df[spend_df['Team']==club]['Total_Arrivals'].values[0]) * pct
    clean_df = player_df[player_df['club'] != club]
    limit_df=clean_df[clean_df['value_in_mil']<spend_limit]
    age_df = limit_df[['club','age']].groupby('club').mean()
    age_df['age'] = round(age_df['age'],2)
    age_df = age_df.reset_index()
    target_age = player_df[player_df['club'] == club]
    target_age=target_age['age'].values[0]
    pos_df = limit_df[player_df['position']==pos_to_fill]
    age_range_df = pos_df[(pos_df['age']>=(target_age-1.5)) & (pos_df['age']<=(target_age+1.5))]
    return(age_range_df.head(1)['short_name'].values[0])

In [ ]:
rec_player_by_pos('Liverpool')

In [162]:
def recommend_players(club,count):
    player_array = []
    if(count == 1):
        player = rec_player_by_role(club,1)
        player_array.append(player)
        print(player_array)
    elif(count == 2):
        player = rec_player_by_role(club,.6)
        player_array.append(player)
        player = rec_player_by_pos(club,.4)
        player_array.append(player)
        print(player_array)
    elif(count == 3):
        player = rec_player_by_role(club,.45)
        player_array.append(player)
        player = rec_player_by_pos(club,.30)
        player_array.append(player)
        player = rec_player_by_role(club,.25)
        player_array.append(player)
        print(player_array)
    elif(count == 4):
        player = rec_player_by_role(club,.40)
        player_array.append(player)
        player = rec_player_by_pos(club,.30)
        player_array.append(player)
        player = rec_player_by_role(club,.20)
        player_array.append(player)
        player = rec_player_by_pos(club,.10)
        player_array.append(player)
        print(player_array)
    elif(count == 5):
        player = rec_player_by_role(club,.35)
        player_array.append(player)
        player = rec_player_by_pos(club,.28)
        player_array.append(player)
        player = rec_player_by_role(club,.21)
        player_array.append(player)
        player = rec_player_by_pos(club,.15)
        player_array.append(player)
        player = rec_player_by_pos(club,.10)
        player_array.append(player)
        print(player_array)

In [165]:
recommend_players('Manchester United',3)

['T. Kroos', 'Rodrigo', 'G. Wijnaldum']


c:\users\akhil_s_bhat\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
player_array

In [195]:
def club_gap_2(club,pr):
    club_tier = (team_rankings[team_rankings['Team'] == club]['Tier']).values[0]
    club_df = player_data[player_data['club']==club][['overall','section','position']]
    club_df = club_df.groupby(['position']).mean()
    club_df = club_df.reset_index()
    upper_tier_df = pos_df[pos_df['Tier']==(club_tier-1)]
    upper_tier_df = upper_tier_df.reset_index()
    upper_tier_df = upper_tier_df.drop(['index','Tier'],axis=1)
    combo_df = pd.merge(upper_tier_df,club_df,how='inner',on='position')
    combo_df.columns = ['Position','Upper_Tier_Rating','Club_Rating']
    combo_df['Rating_Gap'] = combo_df['Upper_Tier_Rating'] - combo_df['Club_Rating']
#     pos_focus = combo_df[combo_df['Rating_Gap']==combo_df['Rating_Gap'].values.max()]['Position'].values[0]
    pos_focus = combo_df[combo_df['Rating_Gap'] == np.sort(combo_df['Rating_Gap'])[-1*pr]]['Position'].values[0]
    return pos_focus

In [196]:
abc = club_gap_2('Liverpool',2)

In [197]:
abc

'LB'

In [186]:
np.sort(abc['Rating_Gap'])

array([-4.25      , -4.        , -1.07619048,  0.5       ,  2.        ,
        3.75      ,  4.46666667,  4.5       , 11.44444444, 11.77777778])

In [185]:
np.sort(abc['Rating_Gap'])[-2]

11.444444444444443

In [189]:
abc[abc['Rating_Gap'] == np.sort(abc['Rating_Gap'])[-1]]

,Position,Upper_Tier_Rating,Club_Rating,Rating_Gap
9,ST,79.111111,67.333333,11.777778
